# TP2: Modelado sobre dataset Fashion MNIST

## Librerías

Fuente: https://pjreddie.com/projects/mnist-in-csv/

In [1]:
import pandas as pd
import os
import gzip
import numpy as np
from tqdm import tqdm
import requests

from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

Correr una sola vez para descargar toda la data luego mutear todo el chunk. 

In [2]:
# def download_gz_from_url(url, output_folder): 
#     file_name = url.split("/")[-1]
#     r = requests.get(url, stream=True)
#     with open(output_folder+"/"+file_name, 'wb') as f:
#         for chunk in r.raw.stream(1024, decode_content=False):
#             if chunk:
#                 f.write(chunk)

# output_folder= "./data"
                
# from pathlib import Path
# Path(output_folder).mkdir(parents=True, exist_ok=True)
                
# urls = ["http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz", #x_train
#         "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz", #y_train
#         "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz", #x_test
#         "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz" #y_test
#        ]
                
# for url in tqdm(urls):
#     download_gz_from_url(url,output_folder)

## Carga de Datos

In [3]:
import mnist_fashion_reader as mnist

X_train, y_train = mnist.load_mnist('data/', kind='train') #devuelve amigables objetos de Pandas
X_test, y_test = mnist.load_mnist('data/', kind='t10k') #devuevle amigables objetos de Pandas

## Punto 1: Exploración del dataset

Realizar una breve descripción del dataset. ¿Qué cantidad de datos tiene este conjunto? ¿Cuántos atributos? ¿Qué rangos de valores toman los atributos? ¿Qué cantidad de clases hay para clasificar? ¿Están balanceadas? Realizar un gráfico donde se muestran ejemplos de las clases (Puede ir a un anexo).

### Cantidad de datos del conjunto

In [4]:
df_entrenamiento = pd.concat([X_train.reset_index(drop=True), y_train], axis=1)
df_test = pd.concat([X_test.reset_index(drop=True), y_test], axis=1)
df_completo = pd.concat([df_entrenamiento, df_test])

In [5]:
print(f"El dataset completo posee {len(df_completo)} registros. Cada registro corresponde a una imagen del dataset.")

El dataset completo posee 70000 registros. Cada registro corresponde a una imagen del dataset.


### Atributos

#### Cantidad de atributos

In [6]:
print(f"Cada atributo representa el valor de cada uno de los pixeles de cada imagen. Dado que cada imagen tiene un formato de 28 x 28 pixeles en escala de grises, el dataset posee {len(df_completo.columns)} atributos por registro, correspondientes al valor en escala de gris de cada pixel que compone la imagen")

Cada atributo representa el valor de cada uno de los pixeles de cada imagen. Dado que cada imagen tiene un formato de 28 x 28 pixeles en escala de grises, el dataset posee 785 atributos por registro, correspondientes al valor en escala de gris de cada pixel que compone la imagen


#### Rangos de valores

In [7]:
max_val = df_completo.max().max()
min_val = df_completo.min().min()

In [8]:
print(f"El valor mínimo de los atributos es {min_val} , y el máximo es {max_val}. Cada valor es un integer entre {min_val} y {max_val}")

El valor mínimo de los atributos es 0 , y el máximo es 255. Cada valor es un integer entre 0 y 255


#### Clases a clasificar

In [9]:
df_completo[0].unique()

array([9, 0, 3, 2, 7, 5, 1, 6, 4, 8], dtype=uint8)

In [10]:
print(f"El dataset está compuesto por {len(df_completo[0].unique())} clases")

El dataset está compuesto por 10 clases


##### N de clases

In [11]:
print(f"Las clases son: {df_completo[0].unique()}")

Las clases son: [9 0 3 2 7 5 1 6 4 8]


##### Balanceo de clases

In [12]:
df_completo[0].value_counts()

9    7000
8    7000
7    7000
6    7000
5    7000
4    7000
3    7000
2    7000
1    7000
0    7000
Name: 0, dtype: int64

In [13]:
(df_completo[0].value_counts())/len(df_completo)

9    0.1
8    0.1
7    0.1
6    0.1
5    0.1
4    0.1
3    0.1
2    0.1
1    0.1
0    0.1
Name: 0, dtype: float64

##### Visualización 

## Separación en desarrollo y testeo

Separación de datos respetando la división presente en el github oficial de desarrollo y testeo

In [14]:
X_train.shape

(60000, 784)

In [15]:
y_train.shape

(60000,)

In [16]:
X_test.shape

(10000, 784)

In [17]:
y_test.shape

(10000,)

## Separación del conjunto de desarrollo en entrenamiento y validación

x_train, y_train = 80% del conjunto de entrenamiento
x_val, y_val = 20% del conjunto de entrenamiento
X_test, y_test = Conjunto held-out de testing

In [18]:
from sklearn.model_selection import train_test_split 
x_train, x_val, y_train, y_val = train_test_split(X_train,y_train, test_size=0.2) #Desarrollo 85% (Train 0.8, validación 0.20) y test 15% 

## Entrenamiento inicial de modelos

### Boosting

#### GradientBoostingClassifier

In [ ]:
gbc = GradientBoostingClassifier(random_state=42)
gbc.fit(x_train, y_train)

print("Accuracy training : {:.3f}".format(gbc.score(x_train, y_train)))
print("Accuracy val: {:.3f}".format(gbc.score(x_val, y_val)))

### Bagging

#### Random Forest Classifier

In [21]:
rf = RandomForestClassifier(random_state=42)
rf.fit(x_train, y_train)

print("Accuracy training : {:.3f}".format(rf.score(x_train, y_train)))
print("Accuracy val: {:.3f}".format(rf.score(x_val, y_val)))

Accuracy training : 1.000
Accuracy val: 0.880


### Árbol de decisión

In [ ]:
ad = DecisionTreeClassifier(random_state=42)
ad.fit(x_train, y_train)

print("Accuracy training : {:.3f}".format(ad.score(x_train, y_train)))
print("Accuracy val: {:.3f}".format(ad.score(x_val, y_val)))

### Naive Bayes

In [ ]:
nb = MultinomialNB(random_state=42)
nb.fit(x_train, y_train)

print("Accuracy training : {:.3f}".format(nb.score(x_train, y_train)))
print("Accuracy val: {:.3f}".format(nb.score(x_val, y_val)))

## Exploración de modelo Random Forest

## Exploración de modelo Gradient Boosting Machine

## Análisis de mejores modelos obtenidos de Random Forest y Gradient Boosting Machine

### Evaluación con conjunto de test

### Análisis de resultados en base a métricas elegidas

#### Análisis de métricas

### Conclusión sobre mejor modelo

### Matriz de confusión de mejor modelo

## Estabilidad del modelo frente al ruido